© Alexander Jüstel, Fraunhofer IEG, Institution for Energy Infrastructures and Geothermal Systems, RWTH Aachen University,
GNU Lesser General Public License v3.0


This notebook was created by Alexander Jüstel and is part of the DGE Rollout Heat Demand Mapping project carried out by students of RWTH Aachen University within the framework of their master thesis. 


# Statistical Evaluation of the commercial Heat demand data

This notebook illustrates the statistical evaluation on a national scale for the commercial heat demand using `rasterstats` and `pandas`.


# Calculating zonal statistics for commercial heat demand

The zonal statistics will be calculated using a shape file containing one polygon for each country and the raster containing the commercial heat demand.

# Importing Libraries

In [1]:
# Importing Libraries
from rasterstats import zonal_stats
import pandas as pd
import geopandas as gpd

In [2]:
# Calculating zonal statistics
stats_com_heat = zonal_stats("../../../data/heat_demand_mapping/shape/Interreg_NWE_Countries_3034.shp", 
                             "../../../data/heat_demand_mapping/tif_results/NWE_HD_tert.tif",
                             stats="count min mean max median sum std",
                             geojson_out=True)


## Storing the properties in a list 

All Properties will be stored in a list. The name of the countries will be stored as well to identify the data set later on. The values for Scotland will be set to zero as no commercial heat demand was reported for this country. 

In [3]:
# Setting all values for Scotland to 0
stats_com_heat[4]['properties']['min'] = 0
stats_com_heat[4]['properties']['max'] = 0
stats_com_heat[4]['properties']['mean'] = 0
stats_com_heat[4]['properties']['count'] = 0
stats_com_heat[4]['properties']['sum'] = 0
stats_com_heat[4]['properties']['std'] = 0
stats_com_heat[4]['properties']['median'] = 0
stats_com_heat[4]['properties']

OrderedDict([('NAME_LATN', 'Scotland'),
             ('NUTS_NAME', 'Scotland'),
             ('area2', 74237),
             ('min', 0),
             ('max', 0),
             ('mean', 0),
             ('count', 0),
             ('sum', 0),
             ('std', 0),
             ('median', 0)])

In [4]:
# Creating list of properties dicts
properties_com_heat = [stats_com_heat[i]['properties'] for i in range(len(stats_com_heat))]
properties_com_heat[:2]

[OrderedDict([('NAME_LATN', 'France'),
              ('NUTS_NAME', 'France'),
              ('area2', 260768),
              ('min', 2.144058942e-06),
              ('max', 94313.487),
              ('mean', 92.80042023891619),
              ('count', 1031671),
              ('sum', 95739502.3483029),
              ('std', 409.4260098209666),
              ('median', 12.388)]),
 OrderedDict([('NAME_LATN', 'Great Britain'),
              ('NUTS_NAME', 'Great Britain'),
              ('area2', 154556),
              ('min', 8.132e-12),
              ('max', 3046.040162528048),
              ('mean', 38.45265033914403),
              ('count', 2652945),
              ('sum', 102012766.45398046),
              ('std', 64.94557221209729),
              ('median', 13.453191184028201)])]

## Creating a Dataframe from dicts

A dataframe containing all statistical values is created. 

In [5]:
# Creating DataFrame from list of dicts
df_com_heat = pd.DataFrame(properties_com_heat, index=range(9))

# Creating a column called 'Region'
df_com_heat['Region'] = df_com_heat['NAME_LATN']


## Cumulated Heat demand of the entire Interreg NWE region

The cumulated heat demand is equal to the `sum` of all country values in `MWh`converted to `TWh`. In addition, the column `sum` is equal to the `Combined/Total HD [TWh]`. The `Share of Total HD [%]` is the `Combined/Total HD [TWh]` divided by the cumulated heat demand and converted to `TWh`.

In [6]:
# Creating a new column called 'Commerical HD [TWh]'
df_com_heat['Commercial HD [TWh]'] = df_com_heat['sum']/1e6

# Calculating the commercial cumulated heat demand of the Interreg NWE Region
df_com_heat_cumulated_hd = round(sum(df_com_heat['Commercial HD [TWh]']),1)

# Calculating the share of national heat demand to the commercial heat demand of the NWE Region
df_com_heat['Share of Commercial HD [%]'] = df_com_heat['Commercial HD [TWh]']/df_com_heat_cumulated_hd*100

print(str(df_com_heat_cumulated_hd) + ' TWh')

557.1 TWh


## Calculating the area that is covered by heat demand squares

The are that is covered by heat is equal to the `count` of pixels of the raster times the resolution (100 m * 100 m) times the conversion factor from m2 to km2. In addition, the `Average of heated area total [MWh/ha]` is equal to `mean`. 

In [7]:
# Calculating the area that is covered by heat demand squares
df_com_heat['Area of heat demand [km2]'] = df_com_heat['count']*100*100*1e-6

# Calculating the total Interreg NWE region where a heat demand was calculated
total_area_of_heat_demand = round(sum(df_com_heat['Area of heat demand [km2]']))

# Creating a new column called 'Average of heated area commercial [MWh/ha]'
df_com_heat['Average of heated area of Commercial HD [MWh/ha]'] = df_com_heat['mean']


print(str(total_area_of_heat_demand) + ' km2')
df_com_heat

79312 km2


,NAME_LATN,NUTS_NAME,area2,min,max,mean,count,sum,std,median,Region,Commercial HD [TWh],Share of Commercial HD [%],Area of heat demand [km2],Average of heated area of Commercial HD [MWh/ha]
0,France,France,260768,2.144059e-06,94313.487000,92.800420,1031671,9.573950e+07,409.426010,12.388000,France,95.739502,17.185335,10316.71,92.800420
1,Great Britain,Great Britain,154556,8.132000e-12,3046.040163,38.452650,2652945,1.020128e+08,64.945572,13.453191,Great Britain,102.012766,18.311392,26529.45,38.452650
2,Ireland,Ireland,65150,1.225298e-08,2268.137290,22.719302,456911,1.038070e+07,60.804759,2.735984,Ireland,10.380699,1.863346,4569.11,22.719302
3,Netherlands,Netherlands,26826,2.962000e-12,12456.355397,106.889313,203944,2.179943e+07,276.325683,23.686381,Netherlands,21.799434,3.913020,2039.44,106.889313
4,Scotland,Scotland,74237,0.000000e+00,0.000000,0.000000,0,0.000000e+00,0.000000,0.000000,Scotland,0.000000,0.000000,0.00,0.000000
5,Schweiz/Suisse/Svizzera,Schweiz/Suisse/Svizzera,38663,1.031770e-10,70218.845336,96.297414,278855,2.685302e+07,431.057871,20.420405,Schweiz/Suisse/Svizzera,26.853015,4.820143,2788.55,96.297414
6,Luxembourg,Luxembourg,2421,5.740837e-06,1229.264055,56.262237,49025,2.758256e+06,91.266176,21.120842,Luxembourg,2.758256,0.495110,490.25,56.262237
7,Germany,Germany,136800,-3.250924e+01,212934.991741,94.270380,2826342,2.664403e+08,782.703739,22.390298,Germany,266.440334,47.826303,28263.42,94.270380
8,Belgium,Belgium,28595,1.825373e-09,200000.000000,72.058297,431487,3.109222e+07,1977.675971,2.294059,Belgium,31.092219,5.581084,4314.87,72.058297


In [8]:
# Calculating the average heat demand of the heated area 
average_of_heated_area_total = sum(df_com_heat['mean'])/9

print(average_of_heated_area_total)

64.4166682358436


In [9]:
# Loading the Interreg NWE Region as Shape File
gdf_interreg_nwe = gpd.read_file("../../../data/heat_demand_mapping/shape/Interreg_NWE_Countries_3034.shp")

# Adding the area im m2 to the gdf
gdf_interreg_nwe['area'] = gdf_interreg_nwe.area

# Converting the area in m2 to km2
gdf_interreg_nwe['Area (planimetric) [km2]'] = round(gdf_interreg_nwe['area']*1e-6)

# Changing the data type of the column to int
gdf_interreg_nwe['Area (planimetric) [km2]'] = gdf_interreg_nwe['Area (planimetric) [km2]'].astype(int)

# Calculating the total planimetric area of the Interreg NWE Region
gdf_interreg_nwe_total_area_planimetric = sum(gdf_interreg_nwe['Area (planimetric) [km2]'])

# Adding the ellipsoidal areas manually to the DataFrame, values were calculated using QGIS
gdf_interreg_nwe['Area (ellipsoidal) [km2]']= [279030,
                                               165486,
                                               69738,
                                               28755,
                                               78684,
                                               41263,
                                               2596,
                                               146607,  
                                               30665]  

# Calculating the total ellipsoidal area of the Interreg NWE Region
gdf_interreg_nwe_total_area_ellipsoid = sum(gdf_interreg_nwe['Area (ellipsoidal) [km2]'])

# Calculating the share of each country to the total ellipsoidal Interreg NWE Region
gdf_interreg_nwe['Share of Total Area [%]'] = gdf_interreg_nwe['Area (ellipsoidal) [km2]']/gdf_interreg_nwe_total_area_ellipsoid*100

print(str(gdf_interreg_nwe_total_area_planimetric) + ' km2')
print(str(gdf_interreg_nwe_total_area_ellipsoid) + ' km2')
gdf_interreg_nwe

788016 km2
842824 km2


,NAME_LATN,NUTS_NAME,area2,geometry,area,Area (planimetric) [km2],Area (ellipsoidal) [km2],Share of Total Area [%]
0,France,France,260768,"MULTIPOLYGON (((2930641.617 2531547.039, 29306...",2.607676e+11,260768,279030,33.106556
1,Great Britain,Great Britain,154556,"MULTIPOLYGON (((2874711.281 2699278.506, 28747...",1.545561e+11,154556,165486,19.634704
2,Ireland,Ireland,65150,"MULTIPOLYGON (((2651794.967 2994393.242, 26518...",6.514953e+10,65150,69738,8.274325
3,Netherlands,Netherlands,26826,"MULTIPOLYGON (((3667957.194 2938854.305, 36680...",2.682582e+10,26826,28755,3.411744
4,Scotland,Scotland,74237,"MULTIPOLYGON (((3075395.957 3245560.731, 30753...",7.423735e+10,74237,78684,9.335757
5,Schweiz/Suisse/Svizzera,Schweiz/Suisse/Svizzera,38663,"POLYGON ((3963266.327 2322163.044, 3963280.662...",3.866341e+10,38663,41263,4.895803
6,Luxembourg,Luxembourg,2421,"POLYGON ((3733429.004 2606067.737, 3733284.924...",2.421410e+09,2421,2596,0.308012
7,Germany,Germany,136800,"MULTIPOLYGON (((3954764.925 2330292.165, 39546...",1.368000e+11,136800,146607,17.394735
8,Belgium,Belgium,28595,"MULTIPOLYGON (((3496569.868 2727532.821, 34976...",2.859470e+10,28595,30665,3.638363


## Calculating Average total heat demand

In [10]:
# Calculating the average commercial heat demand including not heated areas
average_total_heat_demand = df_com_heat_cumulated_hd/gdf_interreg_nwe_total_area_ellipsoid * 1e4

print(str(average_total_heat_demand)+' MWh/ha')

6.609920932484124 MWh/ha


## Merge DataFrames

The two DataFrames are now being merged. In addition, a row for the values of the entire Interreg NWE Region is added.

In [11]:
# Merging DataFrames
gdf_com_heat = pd.merge(df_com_heat, gdf_interreg_nwe, 
                        on=['NUTS_NAME', 'area2'], 
                        how='left')

# Calculating the average HD demand per hectare including areas that are not heated 
gdf_com_heat['Average of Commercial HD [MWh/ha]'] = gdf_com_heat['Commercial HD [TWh]']*1e6/gdf_com_heat['Area (ellipsoidal) [km2]']/100

# Calculating the share of heated heat demand area to the total area
gdf_com_heat['Share of heated heat demand area [%]'] = gdf_com_heat['Area of heat demand [km2]']/gdf_com_heat['Area (ellipsoidal) [km2]']*100

# Creating a dict summarizing the the calculated values for the entire Interreg NWE Region
total_interreg = {'Region': 'Interreg NWE', 
                  'Area (planimetric) [km2]': gdf_interreg_nwe_total_area_planimetric,
                  'Area (ellipsoidal) [km2]': gdf_interreg_nwe_total_area_ellipsoid,
                  'Share of Total Area [%]': 100, 
                  'Commercial HD [TWh]': df_com_heat_cumulated_hd,
                  'Share of Commercial HD [%]': 100, 
                  'Average of Commercial HD [MWh/ha]': average_total_heat_demand,
                  'Average of heated area of Commercial HD [MWh/ha]': average_of_heated_area_total, 
                  'Area of heat demand [km2]': total_area_of_heat_demand,
                  'Share of heated heat demand area [%]': total_area_of_heat_demand/gdf_interreg_nwe_total_area_ellipsoid*100}

# Appending the row to the DataFrame
gdf_com_heat = gdf_com_heat.append(total_interreg,ignore_index=True)

# Selecting columns
gdf_com_heat = gdf_com_heat[['Region', 'Area (planimetric) [km2]', 
                             'Area (ellipsoidal) [km2]', 
                             'Share of Total Area [%]', 
                             'Commercial HD [TWh]', 
                             'Share of Commercial HD [%]', 
                             'Average of Commercial HD [MWh/ha]', 
                             'Average of heated area of Commercial HD [MWh/ha]', 
                             'Area of heat demand [km2]',
                             'Share of heated heat demand area [%]']].sort_values(by='Commercial HD [TWh]', ascending=False).reset_index().drop('index', axis=1)
gdf_com_heat.round(decimals=1)#.style.applymap(lambda x: "background-color: red" if x>0 else "background-color: green")

,Region,Area (planimetric) [km2],Area (ellipsoidal) [km2],Share of Total Area [%],Commercial HD [TWh],Share of Commercial HD [%],Average of Commercial HD [MWh/ha],Average of heated area of Commercial HD [MWh/ha],Area of heat demand [km2],Share of heated heat demand area [%]
0,Interreg NWE,788016,842824,100.0,557.1,100.0,6.6,64.4,79312.0,9.4
1,Germany,136800,146607,17.4,266.4,47.8,18.2,94.3,28263.4,19.3
2,Great Britain,154556,165486,19.6,102.0,18.3,6.2,38.5,26529.4,16.0
3,France,260768,279030,33.1,95.7,17.2,3.4,92.8,10316.7,3.7
4,Belgium,28595,30665,3.6,31.1,5.6,10.1,72.1,4314.9,14.1
5,Schweiz/Suisse/Svizzera,38663,41263,4.9,26.9,4.8,6.5,96.3,2788.5,6.8
6,Netherlands,26826,28755,3.4,21.8,3.9,7.6,106.9,2039.4,7.1
7,Ireland,65150,69738,8.3,10.4,1.9,1.5,22.7,4569.1,6.6
8,Luxembourg,2421,2596,0.3,2.8,0.5,10.6,56.3,490.2,18.9
9,Scotland,74237,78684,9.3,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
gdf_com_heat.round(decimals=1).to_csv('Statistics_Commercial_Heat_Demand.csv', index=False)